In [ ]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)

In [ ]:
#Carregar a tabela de buscas de produtos:
produtos_df = pd.read_excel(r'Base de Buscas\buscas.xlsx')
display(produtos_df)

In [ ]:
#Abrir o site do Google Shopping:
navegador.get("https://shopping.google.com.br/")

In [ ]:
#Buscar o produto:
navegador.find_element(By.ID, 'REsRA').send_keys(produtos_df.loc[0, 'Nome'])
navegador.find_element(By.CLASS_NAME, 'uDgUL').click()

In [ ]:
preco_minimo = float(produtos_df.loc[0, 'Preço mínimo'])
preco_maximo = float(produtos_df.loc[0, 'Preço máximo'])

In [ ]:
#Pegar todos os produtos da página:
lista_produtos = navegador.find_elements(By.CLASS_NAME, 'sh-dgr__content')

for produto in lista_produtos:
    #Obter os dados dos produtos:
    nome_produto = produto.find_element(By.TAG_NAME, 'h3').text.lower().replace(',','')
    preco_produto = produto.find_element(By.CLASS_NAME, 'kHxwFf').text.split(" ")
    preco_produto = preco_produto[1].replace('\nR$', '').replace('.', '').replace(',', '.')
    
    #Filtrar pelos dados da busca:
    lista_termos_produto = produtos_df.loc[0, 'Nome'].lower().split(" ")
    lista_termos_banidos = produtos_df.loc[0, 'Termos banidos'].lower().split(" ")

    termo_prod_correto = all(termo in nome_produto for termo in lista_termos_produto)

    termo_banido_encontrado = any(termo in nome_produto for termo in lista_termos_banidos)

    if termo_prod_correto and not termo_banido_encontrado:
        if preco_minimo <= float(preco_produto) <= preco_maximo: